In [1]:
# -*- coding: utf-8 -*-
"""
Quantum-walk (SamBa-GQW, X-mixer) seeded Memetic Tabu Search (MTS) for LABS

Final output:
  - Best ±1 LABS sequence of length N
  - Best LABS energy

Notes:
  - Quantum-walk seed generation uses a statevector of size 2^N (memory heavy).
  - Practical N is small (e.g., <= 18~20 depending on your machine).
"""

from __future__ import annotations
from dataclasses import dataclass
from typing import Callable, Dict, List, Optional, Tuple
import numpy as np
import random

# ============================================================
# 0) Bit / int utilities (big-endian bit order)
# ============================================================

def int_to_bitstring(i: int, n: int) -> np.ndarray:
    """Integer -> bitstring (0/1) length n, big-endian."""
    bits = np.empty(n, dtype=np.uint8)
    for j in range(n):
        bits[n - j - 1] = (i >> j) & 1
    return bits

def bitstring_to_int(bits: np.ndarray) -> int:
    """Bitstring (0/1) -> integer (big-endian)."""
    out = 0
    n = bits.size
    for j in range(n):
        out |= int(bits[n - j - 1]) << j
    return out

def hamming_neighbors(bitvec01: np.ndarray) -> List[int]:
    """Hypercube neighbors for X-mixer (1-bit flip). Returns neighbor indices as ints."""
    n = bitvec01.size
    out = []
    for i in range(n):
        nb = bitvec01.copy()
        nb[i] = 1 - nb[i]
        out.append(bitstring_to_int(nb))
    return out

# ============================================================
# 1) LABS energy (±1 sequence)
# ============================================================

def labs_energy_pm1(seq_pm1: np.ndarray) -> float:
    """
    LABS energy for ±1 sequence s of length N:
      E(s) = sum_{k=1}^{N-1} C_k(s)^2
      C_k(s) = sum_{i=0}^{N-k-1} s[i]*s[i+k]
    """
    s = seq_pm1.astype(np.int8)
    N = s.size
    E = 0
    for k in range(1, N):
        Ck = int(np.dot(s[:-k], s[k:]))
        E += Ck * Ck
    return float(E)

def labs_energy_bitvec(bitvec01: np.ndarray) -> float:
    """
    Convert 0/1 bitvector to ±1 sequence via:
      0 -> -1, 1 -> +1
    then compute LABS energy.
    """
    seq_pm1 = np.where(bitvec01.astype(np.int8) == 1, 1, -1)
    return labs_energy_pm1(seq_pm1)

def labs_symmetry_int(x_int: int, n: int) -> List[int]:
    """
    Optional symmetry pruning for sampling (helps reduce duplicates):
      - global spin flip: b -> 1-b
      - reversal: reverse bits
      - reversal + flip
    Returns symmetric states as integers.
    """
    b = int_to_bitstring(x_int, n)
    rev = b[::-1]
    flip = 1 - b
    rev_flip = 1 - rev
    return [
        bitstring_to_int(flip),
        bitstring_to_int(rev),
        bitstring_to_int(rev_flip),
    ]

# ============================================================
# 2) SamBa-GQW offline Sampler (Algorithm 2) -> ES (gap list)
# ============================================================

def sampler_alg2_ES(
    *,
    n: int,
    q: int,
    cost_int: Callable[[int], float],
    neighbors_fun: Callable[[np.ndarray], List[int]],
    local_mixer_gap: float = 2.0,
    seed: int = 0,
    use_symmetry: bool = True,
) -> List[float]:
    """
    Paper-faithful Algorithm 2 (SAMPLER):
      - sample q candidates x
      - compute Δ = {(C(x)-C(y))/|Δ^M| : C(x)>C(y), y in neighbors(x)}
      - update running mean per energy level C(x) with max Δ
      - ES <- values(E); add 0; dedup; sort decreasing
    """
    rng = random.Random(seed)

    mean_by_cost: Dict[float, float] = {}
    cnt_by_cost: Dict[float, int] = {}
    forbidden: set[int] = set()

    def online_mean(prev_mean: float, prev_cnt: int, new_val: float) -> float:
        return prev_mean + (new_val - prev_mean) / float(prev_cnt + 1)

    accepted = 0
    while accepted < q:
        x = rng.getrandbits(n)
        if x in forbidden:
            continue

        if use_symmetry:
            forbidden.update(labs_symmetry_int(x, n))
        forbidden.add(x)

        xb = int_to_bitstring(x, n)
        neigh = neighbors_fun(xb)

        Cx = float(cost_int(x))
        deltas = []
        for y in neigh:
            Cy = float(cost_int(y))
            if Cx > Cy:
                deltas.append((Cx - Cy) / float(local_mixer_gap))

        if not deltas:
            continue

        max_delta = float(max(deltas))
        prev_m = mean_by_cost.get(Cx, 0.0)
        prev_c = cnt_by_cost.get(Cx, 0)
        mean_by_cost[Cx] = online_mean(prev_m, prev_c, max_delta)
        cnt_by_cost[Cx] = prev_c + 1

        accepted += 1

    ES = list(mean_by_cost.values())
    ES.append(0.0)
    ES = sorted(set(float(v) for v in ES), reverse=True)
    return ES

# ============================================================
# 3) Online part (3.4 Implementation): build layer schedule
# ============================================================

def make_layer_schedule_from_ES(
    *,
    ES_desc: List[float],
    p_list: List[int],
) -> List[Tuple[float, float]]:
    """
    Build the QAOA-like (but non-optimized) layer schedule from ES:
      intervals correspond to ES[l] -> ES[l+1]
      tau_l = (pi/(2sqrt2)) * 1/Gamma_l
      Gamma_{l,r} = Gamma_l - (r+1/2)*(Gamma_l - Gamma_{l+1})/p_l
      Each layer uses:
        dt_cost  = tau_l/p_l
        dt_mixer = dt_cost * Gamma_{l,r}
    Returns list of (dt_cost, dt_mixer).
    """
    if len(ES_desc) < 2 or ES_desc[-1] != 0.0:
        raise ValueError("ES must contain >=2 values and end with 0.0.")
    if len(p_list) != len(ES_desc) - 1:
        raise ValueError("p_list length must be |ES|-1.")

    layers: List[Tuple[float, float]] = []
    const = float(np.pi / (2.0 * np.sqrt(2.0)))

    for l in range(len(ES_desc) - 1):
        Gamma_l = float(ES_desc[l])
        Gamma_lp1 = float(ES_desc[l + 1])
        p_l = int(p_list[l])

        if Gamma_l <= 0.0:
            continue

        tau_l = const * (1.0 / Gamma_l)
        for r in range(1, p_l + 1):
            Gamma_lr = Gamma_l - (r + 0.5) * ((Gamma_l - Gamma_lp1) / p_l)
            dt_cost = tau_l / p_l
            dt_mixer = dt_cost * Gamma_lr
            layers.append((dt_cost, dt_mixer))

    return layers

# ============================================================
# 4) X-mixer statevector evolution without building 2^N matrices
# ============================================================

def apply_cost_phase_inplace(state: np.ndarray, phases: np.ndarray) -> None:
    state *= phases

def apply_rx_all_qubits(state: np.ndarray, n: int, theta: float) -> np.ndarray:
    """
    Apply ⊗ RX(theta) to a statevector (size 2^n), big-endian qubit order.
    RX(theta) = [[cos(theta/2), -i sin(theta/2)],
                 [-i sin(theta/2), cos(theta/2)]]
    """
    c = np.cos(theta / 2.0)
    s = -1j * np.sin(theta / 2.0)

    out = state.copy()
    dim = out.size
    for q in range(n):
        stride = 1 << (n - 1 - q)
        block = stride << 1
        for base in range(0, dim, block):
            for i in range(stride):
                i0 = base + i
                i1 = i0 + stride
                a0 = out[i0]
                a1 = out[i1]
                out[i0] = c * a0 + s * a1
                out[i1] = s * a0 + c * a1
    return out

def evolve_layers_xmixer_statevector(
    *,
    n: int,
    cost_vec: np.ndarray,                  # length 2^n
    layers: List[Tuple[float, float]],
    initial_state: Optional[np.ndarray] = None,
) -> np.ndarray:
    dim = 2 ** n
    if initial_state is None:
        state = np.ones(dim, dtype=np.complex128) / np.sqrt(dim)
    else:
        state = initial_state.astype(np.complex128, copy=True)
        state = state / np.linalg.norm(state)

    for dt_cost, dt_mixer in layers:
        phases = np.exp(-1j * dt_cost * cost_vec)     # cost phase
        apply_cost_phase_inplace(state, phases)

        # X-mixer: H_M = -ΣX so exp(-i dt_mixer H_M) = ⊗ RX(2 dt_mixer)
        state = apply_rx_all_qubits(state, n, theta=2.0 * dt_mixer)

    return state

def probs_from_state(state: np.ndarray) -> np.ndarray:
    p = np.abs(state) ** 2
    p = np.maximum(p, 0.0)
    return p / p.sum()

# ============================================================
# 5) Quantum-walk initial population generator (P_Q)
# ============================================================

@dataclass(frozen=True)
class PQResult:
    population_pm1: List[np.ndarray]          # K sequences (±1) length N
    threshold_energy: float
    ES: List[float]
    layers: List[Tuple[float, float]]

def samba_gqw_xmixer_initial_population_for_LABS(
    *,
    N: int,
    K: int,
    elite_frac: float = 0.05,
    q: Optional[int] = None,              # default N^2
    p_per_interval: int = 4,
    shots: Optional[int] = None,
    seed: int = 0,
    use_symmetry: bool = True,
) -> PQResult:
    rng = np.random.default_rng(seed)
    q_eff = q if q is not None else N * N

    def cost_int(x: int) -> float:
        return labs_energy_bitvec(int_to_bitstring(x, N))

    # (A) Offline sampler -> ES
    ES = sampler_alg2_ES(
        n=N,
        q=q_eff,
        cost_int=cost_int,
        neighbors_fun=hamming_neighbors,
        local_mixer_gap=2.0,
        seed=seed,
        use_symmetry=use_symmetry,
    )

    # (B) Build layer schedule
    p_list = [p_per_interval] * (len(ES) - 1)
    layers = make_layer_schedule_from_ES(ES_desc=ES, p_list=p_list)

    # (C) Build full cost vector (diagonal H_C)
    dim = 2 ** N
    cost_vec = np.empty(dim, dtype=np.float64)
    for x in range(dim):
        cost_vec[x] = cost_int(x)

    # (D) Evolve by layers
    stateT = evolve_layers_xmixer_statevector(n=N, cost_vec=cost_vec, layers=layers)
    probs = probs_from_state(stateT)

    # (E) Sample measurements
    if shots is None:
        shots = int(np.ceil(K / elite_frac) * 3)

    idx_samples = rng.choice(dim, size=shots, p=probs)
    samples01 = np.stack([int_to_bitstring(int(i), N) for i in idx_samples], axis=0)

    energies = np.array([labs_energy_bitvec(s) for s in samples01], dtype=float)
    order = np.argsort(energies)
    elite_count = max(1, int(np.ceil(len(order) * elite_frac)))
    threshold = float(energies[order[elite_count - 1]])

    # unique elites first
    seen = set()
    population: List[np.ndarray] = []
    for ix in order[:elite_count]:
        b = samples01[ix]
        key = tuple(int(v) for v in b.tolist())
        if key in seen:
            continue
        seen.add(key)
        population.append(np.where(b == 1, 1, -1).astype(np.int8))
        if len(population) >= K:
            break

    # pad with random if needed
    while len(population) < K:
        b = rng.integers(0, 2, size=N, dtype=np.uint8)
        population.append(np.where(b == 1, 1, -1).astype(np.int8))

    return PQResult(
        population_pm1=population[:K],
        threshold_energy=threshold,
        ES=ES,
        layers=layers,
    )

# ============================================================
# 6) Classical components: combine, mutate, tabu search
# ============================================================

def combine_uniform(p1: np.ndarray, p2: np.ndarray, rng: np.random.Generator) -> np.ndarray:
    mask = rng.random(len(p1)) < 0.5
    return np.where(mask, p1, p2).astype(np.int8)

def mutate_pm1(x: np.ndarray, p_mut: float, rng: np.random.Generator) -> np.ndarray:
    y = x.copy()
    flip = rng.random(len(y)) < p_mut
    y[flip] *= -1
    return y

def tabu_search_labs(
    x0: np.ndarray,
    *,
    tabu_tenure: int = 50,
    max_iter: int = 500,
) -> np.ndarray:
    """
    Simple Tabu Search:
      - neighborhood: single spin flips
      - tabu list on flip indices
    """
    n = len(x0)
    best = x0.copy()
    best_E = labs_energy_pm1(best)

    cur = x0.copy()
    tabu: Dict[int, int] = {}

    for _ in range(max_iter):
        best_move = None
        best_candidate = None
        best_candidate_E = float("inf")

        for i in range(n):
            if tabu.get(i, 0) > 0:
                continue
            cand = cur.copy()
            cand[i] *= -1
            Ec = labs_energy_pm1(cand)
            if Ec < best_candidate_E:
                best_candidate_E = Ec
                best_candidate = cand
                best_move = i

        if best_candidate is None or best_move is None:
            break

        cur = best_candidate

        # decay tabu
        tabu = {k: v - 1 for k, v in tabu.items() if v > 1}
        tabu[best_move] = tabu_tenure

        if best_candidate_E < best_E:
            best = cur.copy()
            best_E = best_candidate_E

    return best

# ============================================================
# 7) Quantum-enhanced MTS (Algorithm 1 skeleton)
# ============================================================

@dataclass(frozen=True)
class MTSResult:
    best_seq_pm1: np.ndarray
    best_energy: float

def quantum_walk_seeded_mts_labs(
    *,
    N: int,
    K: int,
    G_max: int = 1000,
    p_comb: float = 0.7,
    p_mut: float = 0.01,
    elite_frac: float = 0.05,
    seed: int = 0,
    # quantum-walk params
    q: Optional[int] = None,
    p_per_interval: int = 4,
    shots: Optional[int] = None,
    use_symmetry: bool = True,
    # tabu params
    tabu_tenure: int = 50,
    tabu_iters: int = 500,
) -> MTSResult:
    rng = np.random.default_rng(seed)
    random.seed(seed)

    # 1) Initialize population via quantum walk (P_Q)
    pq = samba_gqw_xmixer_initial_population_for_LABS(
        N=N, K=K,
        elite_frac=elite_frac,
        q=q,
        p_per_interval=p_per_interval,
        shots=shots,
        seed=seed,
        use_symmetry=use_symmetry,
    )
    population = [ind.copy() for ind in pq.population_pm1]

    # best so far
    best_seq = min(population, key=labs_energy_pm1)
    best_E = labs_energy_pm1(best_seq)

    # 2) Main loop
    for _ in range(G_max):
        if rng.random() < p_comb:
            p1, p2 = rng.choice(K, size=2, replace=False)
            child = combine_uniform(population[p1], population[p2], rng)
        else:
            child = population[int(rng.integers(0, K))].copy()

        child = mutate_pm1(child, p_mut, rng)

        # memetic step: tabu search
        child = tabu_search_labs(child, tabu_tenure=tabu_tenure, max_iter=tabu_iters)

        Ec = labs_energy_pm1(child)
        if Ec < best_E:
            best_seq = child.copy()
            best_E = Ec

        # replace a random individual
        population[int(rng.integers(0, K))] = child

    return MTSResult(best_seq_pm1=best_seq, best_energy=best_E)

# ============================================================
# 8) Main: run for a given N and print best sequence + energy
# ============================================================

def main():
    # ---- You can change N here ----
    N = 16

    # Population size K for MTS
    K = 200

    # Run
    res = quantum_walk_seeded_mts_labs(
        N=N,
        K=K,
        G_max=1000,
        p_comb=0.7,
        p_mut=0.01,
        elite_frac=0.05,
        seed=42,
        q=None,                # default N^2
        p_per_interval=4,
        shots=None,
        use_symmetry=True,
        tabu_tenure=50,
        tabu_iters=300,
    )

    print("\n=== RESULT ===")
    print("N =", N)
    print("Best LABS energy =", res.best_energy)
    print("Best sequence (±1) =", res.best_seq_pm1.tolist())

if __name__ == "__main__":
    main()



=== RESULT ===
N = 16
Best LABS energy = 24.0
Best sequence (±1) = [-1, -1, 1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1]
